Part B

Instructions are given in <span style="color:blue">blue</span> color.
(Unfortunately, Google Colab won't display the blue color.)

**Note**: This task is a continuation of task A. Please make sure that you have completed it in advance. 

## 4. Creating Limericks
---

In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
# force gpu computing, when gpu library is available
USE_GPU = True

In [2]:
%%capture
torch.manual_seed(0)

In [3]:
CUDA_AVAILABLE = torch.cuda.is_available()
print(f'CUDA_AVAILABLE = {CUDA_AVAILABLE}')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() and USE_GPU else "cpu")
print(f'Doing inference with the trained networks using {device}')

CUDA_AVAILABLE = True
Doing inference with the trained networks using cuda


In [4]:
# Reading data
limericks_file = "Data/limericks.txt"
limericks = open(limericks_file, 'r').read()
vocab = sorted(set(limericks))

In [5]:
# Creating unique index for each character in the text
char_to_index = {char:index for index, char in enumerate(vocab)}
# Mapping the index back to the characters
index_to_char = np.array(vocab)

In [6]:
# Vocabulary length
vocab_size = len(vocab)
# Embedding dimensions
embed_dim = 64
# Number of RNN units (neurons)
rnn_units = 512

* <div style="color:blue">Define the same Network Class as in Task A.</div>
* <div style="color:blue">Load both your trained models into separate model objects and move them to the desired device.</div>

In [7]:
class Network(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_units, recurrent_type='LSTM'):
        super(Network, self).__init__()
        self.recurrent_type = recurrent_type
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if recurrent_type == 'RNN':
            self.rnn = nn.RNN(embedding_dim, rnn_units, batch_first=True)
        elif recurrent_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, rnn_units, batch_first=True)
        elif recurrent_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, rnn_units, batch_first=True)
        self.linear = nn.Linear(rnn_units, vocab_size)
    
    def forward(self, x):
        embedded = self.embedding(x)
        #lstm also returns the cell hidden state
        if self.recurrent_type == "LSTM":
            output, (hidden, cell_hidden) = self.rnn(embedded)
        else:
            output, hidden = self.rnn(embedded)
        output = self.linear(hidden[-1])
        return output

In [8]:
model_v1 = Network(vocab_size, embed_dim, rnn_units, recurrent_type='LSTM')
model_v1.load_state_dict(torch.load("Models/network.pt"))
model_v1.to(device)
model_v2 = Network(vocab_size, embed_dim, rnn_units, recurrent_type='LSTM')
model_v2.load_state_dict(torch.load("Models/network_v2.pt"))
model_v2.to(device)

Network(
  (embedding): Embedding(70, 64)
  (rnn): LSTM(64, 512, batch_first=True)
  (linear): Linear(in_features=512, out_features=70, bias=True)
)

In [9]:
# Helper function to generate text based on a given start sequence
def generate_limericks(model, start_sequence, sequence_length, length=100, temperature=1.0):
    model.eval()
    
    encoded_input = [char_to_index[s] for s in start_sequence]
    encoded_input = torch.tensor(encoded_input).view(1, -1).to(device)

    generated_str = start_sequence
    
    with torch.no_grad():
        for i in range(length):
            logits = model(encoded_input)
            logits = logits.squeeze(0)

            scaled_logits = logits * temperature
            new_char_indx = torch.multinomial(torch.exp(scaled_logits), num_samples=1)

            new_char_indx = new_char_indx[-1].item()

            generated_str += str(index_to_char[new_char_indx])

            new_char_indx = torch.tensor([new_char_indx]).view(1, 1).to(device)
            encoded_input = torch.cat(
                [encoded_input, new_char_indx],
                dim=1)
            encoded_input = encoded_input[:, -sequence_length:]

    return generated_str

* <div style="color:blue">Use your models and the provided helper function <code>generate_limericks</code> to create some text. Evaluate the quality of both models. Were they able to replicate the structure of a limerick? What about rhyme schemes, grammar, and content?</br>
<b>Hint:</b> You might want to play around with the <code>length</code> parameter when generating the limericks</div>

In [10]:
print(generate_limericks(model_v1, "from the mornings", 60, length=200))

from the mornings:
he's a prade of the wart!
i'm a braggle's fhawly, but to rail.
it here's takes bo jonet,
sof a mans he have comprengations,
sumery is the persoze in underd.

uftern the aisus only, crocks as his dui


In [22]:
print(generate_limericks(model_v2, "from the mornings", 60, length=1000, temperature=1.5))

from the mornings,
was ans clearing a starting the back.
but the courtrips should never set.
i am troubles flowers, to call "cow,
at you're taking off and a wide;
on the critimes, make you're cace them call," says he strong.

there once was a figure mom's severe,
they clue down the opening break
than a colon atome word!
it is sure things he'd been the bones.

an amara croop, self-won't corned
to a strught up, i'm earling thicks.
in no mates on emotion
and fresh them out; my divine,
while the audus still ways:
i've been found the divine,
not a buildifi may
excell and starts them all chicken money-
he'd be noans on embroar and spine.

to citious holding about,
it's a profusting around (it ?
my moon, the ole-type god's show!
in the soul was provide to a crime.
give the store of a practive didn't bell!"
"you can see a searlict matter.
though the season with stops,
may be caused his great piece of preciss.

"let no limerick (i'd pressed "but your crimbed by the small.

have a minuter-but kn

The model has obviously been able to learn some of the structure of a limerick, although it has not yet gotten that there are always five lines. The TLM cannot produce sentences which make sense, and it has not picked up on rhyming either (might be something for LLMs).